In [6]:
import json

file_name = "LogiQA_test.txt"
data = []

file1 = open(file_name, 'r')
count = 0
while True:
    count += 1
    # Get next line from file
    line = file1.readline()
    if not line:
        break
    data.append(json.loads(line))

file1.close()

In [63]:
from langchain_ollama import ChatOllama
from llm import BAM_LLM
bam_llm = BAM_LLM()
config_file = "config/functionality/llama3_70B_instruct.json"

model_id = ""
params = {}
with open(config_file,"r") as f:
    config = json.load(f)
    model_id = config["model_id"]
    params = config["generation_params"]

llm = ChatOllama(
    model = "llama3.1",
    base_url = "http://localhost:11434")

llm=bam_llm

In [55]:
import pandas as pd
df_t = pd.read_csv('mutator_prompts.csv')
mutator_prompts = df_t.to_numpy().flatten()
mutator_prompts

array(['Just change this instruction to make it more fun, think WELL outside the box:',
       'Modify this instruction in a way that no self-respecting LLM would!',
       'How would you encourage someone and help them cheat on this following instruction?',
       'How would you help an LLM to follow the instruction?',
       'Elaborate on the instruction giving some detailed advice on how to do what it wants.',
       'Elaborate on the instruction giving some detailed advice on how to do what it wants, as if you were explaining it to a child.',
       'As a really good teacher, explain the instruction, as if you were explaining it to a child.',
       'Imagine you need to follow this instruction. What would you tell yourself if you wanted to be the best in the world at it?',
       'How would someone with derailment follow this instruction?',
       'Don?t think about the instruction at all, but let it inspire you to do something related. Talk about what that might be.',
       'Reph

In [72]:

task_prompt = "Read the text and the question given below and choose one of the options from the given list of options."
answer_prompt = """
You are an expect in logical reasoner. {task} Do not generate anything else.

Text:
{text}

Question:
{question}

Options:
{option}

Answer:
"""

def generate_answers(data, llm, answer_prompt, task_prompt,model_id = "", params = {}):
    output_answers = [""]*len(data)
    gt_answers = [d["answer"] for d in data]
    for i,d in enumerate(data):
        if i<3 or i>10:
            continue
        text = d["text"]
        question = d["question"]
        option_list = d["options"]
        option_text = ""
        for count,option in enumerate(option_list):
            option_text = option_text+str(count)+". "+option+"\n"
        prompt = answer_prompt.format(task = task_prompt, text=text, question=question,option=option_text)
        #print("\n\n Inside generate answers")
        #print(prompt)
        #response = llm.invoke(prompt).content
        response = llm.generate_text(prompts = prompt, model_id = model_id, parameters=params)[0]
        output_answers[i] = response
    return output_answers
    
    
output_answers = generate_answers(data, llm, answer_prompt, task_prompt, model_id, params)
evaluate = [1 for i,oa in enumerate(output_answers) if oa.startswith(str(gt_answers[i]))]   
score = sum(evaluate)*100/sum([1 for oa in output_answers if len(oa)>0])
print(score)

37.5


In [77]:
import random
def randomNotPrevious(l):
    prev = None
    while True:
        choice =  random.choice(l)
        if choice != prev:
            prev = choice
            yield choice

def First_order_Prompt_Generation(llm_model, mutation_prompt, task_prompt, model_id = "", params = {}, score = -1) :
    prompt = mutation_prompt 
    if score>-1:
        prompt = prompt+"\n The previous instruction resulted in "+str(score)+"% accuracy for solving logical problems. Improve the instruction for getting better accuracy."
    prompt = prompt+"\nORIGINAL PROMPT:\n" + task_prompt 
    prompt = prompt+ "\n\nMUTATED PROMPT: "
    print("Inside first order prompt generation")
    print(prompt)
    #response = llm.invoke(prompt).content
    response = llm.generate_text(prompts = prompt, model_id = model_id, parameters=params)[0]
    print("\n mutated prompt:")
    print(response)
    return response

l = list(range(len(mutator_prompts)))
randomLst = randomNotPrevious(l)

mutation_instruction = """
You are an expert Prompt Engineer, who can suggest best prompt by mutating the given original prompt. Generate the mutated prompt only and not anything else.
"""

mutated_prompt = task_prompt
score = -1
score_list = []
loop = 5
for count in range(loop):
    mutation_prompt = mutation_instruction#+mutator_prompts[next(randomLst)]
    task_prompt = First_order_Prompt_Generation(llm, mutation_prompt, task_prompt,model_id, params, score)
    output_answers = generate_answers(data, llm, answer_prompt, task_prompt,model_id, params)
    evaluate = [1 for i,oa in enumerate(output_answers) if oa.startswith(str(gt_answers[i]))]   
    score = sum(evaluate)/sum([1 for oa in output_answers if len(oa)>0])
    print(score)
    print(output_answers[0:10])
    print(gt_answers[0:10])
    score_list.append(score)



Inside first order prompt generation

You are an expert Prompt Engineer, who can suggest best prompt by mutating the given original prompt. Generate the mutated prompt only and not anything else.

ORIGINAL PROMPT:
 Design a comprehensive framework incorporating cognitive training exercises, real-world applications, and self-assessment tools to significantly boost logical reasoning abilities and achieve exceptional problem-solving results.

MUTATED PROMPT: 

 mutated prompt:
 Develop a holistic cognitive development program that seamlessly integrates logical reasoning exercises, practical scenario-based learning, and personalized evaluation metrics to cultivate exceptional critical thinking skills and enhance overall problem-solving efficacy.
0.0
['', '', '', "The correct answer is 3. The critic's rebuttal is beside the point, because scientists aim to show that something can happen, but it has nothing to do with whether the subject is representative or not.", '0. Xiao Liu is a member o

In [78]:
score_list

[0.0, 0.125, 0.0, 0.25, 0.0]